# Анализ данных с помощью SQL

# Содержание

[Доступ к базе данных](#Доступ-к-базе-данных)

[Исходные данные](#Исходные-данные)
1. [Количество книг, вышедших после 1 января 2000 года](#1.-Количество-книг,-вышедших-после-1-января-2000-года)
2. [Определение количества обзоров и средней оценки для каждой книги](#2.-Определение-количества-обзоров-и-средней-оценки-для-каждой-книги)
3. [Определение издательства, выпустившего наибольшее число книг толще 50 страниц](#3.-Определение-издательства,-выпустившего-наибольшее-число-книг-толще-50-страниц)
4. [Определение автора с самой высокой средней оценкой книг](#4.-Определение-автора-с-самой-высокой-средней-оценкой-книг-(учитываются-только-книги-с-50-и-более-оценками))
5. [Определение среднего количества обзоров от пользователей](#5.-Определение-среднего-количества-обзоров-от-пользователей,-которые-поставили-больше-50-оценок)

# Доступ к базе данных

In [1]:
# импорт библиотек

import pandas as pd
from sqlalchemy import create_engine

# кофигурации для подключения находятся в отдельном файле

from configuration import user
from configuration import password
from configuration import host
from configuration import port
from configuration import database

# установка параметров
db_config = {'user': user, # имя пользователя
 'pwd': password, # пароль
 'host': host,
 'port': port, # порт подключения
 'db': database} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохранение коннектора
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


Можно написать функцию SQL-запроса:

In [2]:
# SQL-запрос через Pandas:
# pd.io.sql.read_sql(query, con = engine) 

def func_query(sql_query):
    return pd.io.sql.read_sql(sql_query, con = engine)

# Исходные данные

In [3]:
# данные о книгах

func_query('SELECT * FROM books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [4]:
# данные об авторах
func_query('SELECT * FROM authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [5]:
# данные об издательствах
func_query('SELECT * FROM publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [6]:
# данные о пользовательских оценках книг
func_query('SELECT * FROM ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [7]:
# данные о пользовательских обзорах на книги
func_query('SELECT * FROM reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


# 1. Количество книг, вышедших после 1 января 2000 года

In [8]:
func_query("SELECT COUNT (*) FROM books WHERE publication_date > '2000-01-01' ;")

,count
0,819


# 2. Определение количества обзоров и средней оценки для каждой книги

Определение средней оценки по id книги:

In [9]:
"""
SELECT AVG(rating) AS avg_rating,
book_id
FROM ratings
GROUP BY book_id
ORDER BY avg_rating
"""
func_query('SELECT book_id, AVG(rating) AS avg_rating FROM ratings GROUP BY book_id ORDER BY avg_rating')

,book_id,avg_rating
0,303,1.50
1,202,2.00
2,371,2.00
3,316,2.00
4,915,2.25
...,...,...
995,182,5.00
996,599,5.00
997,62,5.00
998,913,5.00


Определение числа обзоров по id книги:

In [10]:
func_query(
    '''
SELECT COUNT(review_id) AS reviews_count, book_id
FROM reviews
GROUP BY book_id
ORDER BY reviews_count
'''
)

,reviews_count,book_id
0,1,129
1,1,187
2,1,156
3,1,760
4,1,248
...,...,...
989,6,656
990,6,733
991,6,734
992,6,207


Определение средней оценки по названию книги:

In [11]:
func_query(
    '''
SELECT AVG(rating) as avg_rating, title

    FROM (SELECT *
        FROM ratings

        INNER JOIN books ON ratings.book_id = books.book_id) AS SQBT
    GROUP BY title
    ORDER BY avg_rating
'''
)

,avg_rating,title
0,1.50,Harvesting the Heart
1,2.00,Junky
2,2.00,Drowning Ruth
3,2.00,His Excellency: George Washington
4,2.25,The World Is Flat: A Brief History of the Twen...
...,...,...
994,5.00,School's Out—Forever (Maximum Ride #2)
995,5.00,Act of Treason (Mitch Rapp #9)
996,5.00,Wherever You Go There You Are: Mindfulness Me...
997,5.00,Marvel 1602


Определение числа обзоров по названию книги:

In [12]:
func_query(
    '''
SELECT COUNT(review_id) AS review_count, title

    FROM (SELECT *
        FROM reviews

        INNER JOIN books ON reviews.book_id = books.book_id) AS SQBT
    GROUP BY title
    ORDER BY review_count
'''
)

,review_count,title
0,1,Everyday Italian: 125 Simple and Delicious Rec...
1,1,Winter Prey (Lucas Davenport #5)
2,1,Naked Empire (Sword of Truth #8)
3,1,Babyville
4,1,Ten Apples Up On Top!
...,...,...
988,6,Harry Potter and the Prisoner of Azkaban (Harr...
989,6,The Road
990,6,Outlander (Outlander #1)
991,7,Twilight (Twilight #1)


Можно заметить разницу в количестве обзоров. Это связано с разным количеством id книг:

In [13]:
# данные о книгах
func_query(
    """SELECT COUNT(DISTINCT(book_id)) AS books_id_total
    FROM books"""
)

,books_id_total
0,1000


In [14]:
# данные об обзорах
func_query(
    """SELECT COUNT(DISTINCT(book_id)) AS books_id_total
    FROM reviews"""
)

,books_id_total
0,994


In [15]:
# данные о пользовательсктх рейтингах
func_query(
    """SELECT COUNT(DISTINCT(book_id)) AS books_id_total
    FROM ratings"""
)

,books_id_total
0,1000


Также для одной книги (с одним названием) может быть несколько id:

In [16]:
func_query(
    """SELECT book_id, title, publisher_id
    FROM books
    WHERE title = 'Memoirs of a Geisha'"""
)

,book_id,title,publisher_id
0,426,Memoirs of a Geisha,241
1,427,Memoirs of a Geisha,311


Это обусловлено разными издательствами, которые выпускали книгу, и годом выпуска.

Значения средней оценки и числа обзоров книг можно представить в виде объединённой таблицы:

In [17]:
func_query(
    '''
SELECT SQBT3.title, avg_rating, review_count
FROM
    (SELECT AVG(rating) as avg_rating, title
    FROM
        (SELECT *
        FROM ratings

        INNER JOIN books ON ratings.book_id = books.book_id) AS SQBT
        GROUP BY title) AS SQBT3
INNER JOIN (
            SELECT COUNT(review_id) AS review_count, title

            FROM
                (SELECT *
                FROM reviews

                INNER JOIN books ON reviews.book_id = books.book_id) AS SQBT2
                GROUP BY title) AS SQBT4 ON SQBT3.title = SQBT4.title
ORDER BY avg_rating DESC
'''
)

,title,avg_rating,review_count
0,School's Out—Forever (Maximum Ride #2),5.00,3
1,The War of Art: Break Through the Blocks & Win...,5.00,2
2,A Woman of Substance (Emma Harte Saga #1),5.00,2
3,Misty of Chincoteague (Misty #1),5.00,2
4,In the Hand of the Goddess (Song of the Liones...,5.00,2
...,...,...,...
988,The World Is Flat: A Brief History of the Twen...,2.25,3
989,Junky,2.00,2
990,His Excellency: George Washington,2.00,2
991,Drowning Ruth,2.00,3


# 3. Определение издательства, выпустившего наибольшее число книг толще 50 страниц

Определение id первых 3 издательств по наибольшему числу книг:

In [18]:
func_query(
    """
    SELECT publisher_id, COUNT(book_id) AS books_count
    FROM books
    WHERE num_pages > 50
    GROUP BY publisher_id
    ORDER BY books_count DESC
    LIMIT 3
    """
)

,publisher_id,books_count
0,212,42
1,309,31
2,116,25


Определение названий первых 3 издательств по наибольшему числу книг:

In [19]:
func_query(
    """
    SELECT publisher, COUNT(book_id) AS books_count
    FROM
        (SELECT *
        FROM publishers
        INNER JOIN books ON publishers.publisher_id = books.publisher_id) AS SQBT
        GROUP BY publisher
        ORDER BY books_count DESC
        LIMIT 3
    """
)

,publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


Больше всего книг выпустило издательство Penguin Books.

# 4. Определение автора с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)

Определение id книг с самой высокой оценкой (при наличии 50 и более оценок):

In [20]:
func_query(
    """
    SELECT book_id, AVG(rating) AS avg_rating, COUNT(rating) AS ratings_count
    FROM
    ratings
    GROUP BY book_id
    HAVING COUNT(rating) > 50 OR COUNT(rating) = 50
    ORDER BY avg_rating DESC
    """
)

,book_id,avg_rating,ratings_count
0,302,4.414634,82
1,722,4.391892,74
2,299,4.287500,80
3,656,4.264151,53
4,300,4.246575,73
5,399,4.192308,52
6,301,4.186667,75
7,750,4.125000,88
8,779,4.080645,62
9,405,3.901408,71


Получение информации о книгах, чьи id были получены:

In [21]:
func_query(
    """
    SELECT *
    FROM
    books
    INNER JOIN (
        SELECT book_id, AVG(rating) AS avg_rating, COUNT(rating) AS ratings_count
        FROM
        ratings
        GROUP BY book_id
        HAVING COUNT(rating) > 50 OR COUNT(rating) = 50)
        AS SQBT ON SQBT.book_id = books.book_id
    ORDER BY avg_rating DESC
    """
)

,book_id,author_id,title,num_pages,publication_date,publisher_id,book_id,avg_rating,ratings_count
0,302,236,Harry Potter and the Prisoner of Azkaban (Harr...,435,2004-05-01,255,302,4.414634,82
1,722,240,The Fellowship of the Ring (The Lord of the Ri...,398,2003-09-05,157,722,4.391892,74
2,299,236,Harry Potter and the Chamber of Secrets (Harry...,341,1999-06-02,24,299,4.287500,80
3,656,402,The Book Thief,552,2006-03-14,9,656,4.264151,53
4,300,236,Harry Potter and the Half-Blood Prince (Harry ...,652,2006-09-16,255,300,4.246575,73
5,399,376,Little Women,449,2004-04-06,267,399,4.192308,52
6,301,236,Harry Potter and the Order of the Phoenix (Har...,870,2004-09-01,255,301,4.186667,75
7,750,240,The Hobbit or There and Back Again,366,2002-08-15,154,750,4.125000,88
8,779,498,The Lightning Thief (Percy Jackson and the Oly...,375,2006-03-01,91,779,4.080645,62
9,405,621,Lord of the Flies,182,1999-10-01,212,405,3.901408,71


Осталось только добавть имена авторов:

In [22]:
func_query(
    """
    SELECT *
    FROM
    (SELECT *
        FROM
        books
        INNER JOIN (
            SELECT book_id, AVG(rating) AS avg_rating, COUNT(rating) AS ratings_count
            FROM
            ratings
            GROUP BY book_id
            HAVING COUNT(rating) > 50 OR COUNT(rating) = 50)
            AS SQBT ON SQBT.book_id = books.book_id)
            AS D_SQBT
    INNER JOIN authors ON D_SQBT.author_id = authors.author_id
    ORDER BY avg_rating DESC
    """
)

,book_id,author_id,title,num_pages,publication_date,publisher_id,book_id,avg_rating,ratings_count,author_id,author
0,302,236,Harry Potter and the Prisoner of Azkaban (Harr...,435,2004-05-01,255,302,4.414634,82,236,J.K. Rowling/Mary GrandPré
1,722,240,The Fellowship of the Ring (The Lord of the Ri...,398,2003-09-05,157,722,4.391892,74,240,J.R.R. Tolkien
2,299,236,Harry Potter and the Chamber of Secrets (Harry...,341,1999-06-02,24,299,4.287500,80,236,J.K. Rowling/Mary GrandPré
3,656,402,The Book Thief,552,2006-03-14,9,656,4.264151,53,402,Markus Zusak/Cao Xuân Việt Khương
4,300,236,Harry Potter and the Half-Blood Prince (Harry ...,652,2006-09-16,255,300,4.246575,73,236,J.K. Rowling/Mary GrandPré
5,399,376,Little Women,449,2004-04-06,267,399,4.192308,52,376,Louisa May Alcott
6,301,236,Harry Potter and the Order of the Phoenix (Har...,870,2004-09-01,255,301,4.186667,75,236,J.K. Rowling/Mary GrandPré
7,750,240,The Hobbit or There and Back Again,366,2002-08-15,154,750,4.125000,88,240,J.R.R. Tolkien
8,779,498,The Lightning Thief (Percy Jackson and the Oly...,375,2006-03-01,91,779,4.080645,62,498,Rick Riordan
9,405,621,Lord of the Flies,182,1999-10-01,212,405,3.901408,71,621,William Golding


Самая высокая средняя оценка за книгу у автора J.K. Rowling/Mary GrandPré.

Если вычислять среднюю оценку для автора, опираясь на полученную таблицу:

In [23]:
func_query(
    """
    SELECT AVG(avg_rating) AS average_rating, author
    FROM
    (SELECT *
        FROM
        books
        INNER JOIN (
            SELECT book_id, AVG(rating) AS avg_rating, COUNT(rating) AS ratings_count
            FROM
            ratings
            GROUP BY book_id
            HAVING COUNT(rating) > 50 OR COUNT(rating) = 50)
            AS SQBT ON SQBT.book_id = books.book_id
            ) AS D_SQBT
    INNER JOIN authors ON D_SQBT.author_id = authors.author_id
    GROUP BY author
    ORDER BY average_rating DESC
    """
)

,average_rating,author
0,4.283844,J.K. Rowling/Mary GrandPré
1,4.264151,Markus Zusak/Cao Xuân Việt Khương
2,4.258446,J.R.R. Tolkien
3,4.192308,Louisa May Alcott
4,4.080645,Rick Riordan
5,3.901408,William Golding
6,3.825581,J.D. Salinger
7,3.789474,Paulo Coelho/Alan R. Clarke/Özdemir İnce
8,3.787879,William Shakespeare/Paul Werstine/Barbara A. M...
9,3.754540,Dan Brown


J.K. Rowling/Mary GrandPré остаётся автором с самой высокой средней оценкой книг (в данном случае осреднённой средней оценкой).

Можно посчитать среднюю оценку всех книг автора (с 50 и более оценок), учитывая каждый отзыв:

In [24]:
func_query(
    """
    SELECT *
    FROM
    (SELECT *
        FROM
        books
        INNER JOIN (SELECT book_id, rating
        FROM
        ratings
        WHERE book_id IN (SELECT book_id
                        FROM
                        ratings
                        GROUP BY book_id
                        HAVING COUNT(rating) > 50 OR COUNT(rating) = 50)) AS SQBT ON SQBT.book_id = books.book_id
        
    ) AS D_SQBT
    INNER JOIN authors ON D_SQBT.author_id = authors.author_id
   
    """
)

,book_id,author_id,title,num_pages,publication_date,publisher_id,book_id,rating,author_id,author
0,75,106,Angels & Demons (Robert Langdon #1),736,2006-04-01,229,75,5,106,Dan Brown
1,75,106,Angels & Demons (Robert Langdon #1),736,2006-04-01,229,75,4,106,Dan Brown
2,75,106,Angels & Demons (Robert Langdon #1),736,2006-04-01,229,75,4,106,Dan Brown
3,75,106,Angels & Demons (Robert Langdon #1),736,2006-04-01,229,75,4,106,Dan Brown
4,75,106,Angels & Demons (Robert Langdon #1),736,2006-04-01,229,75,4,106,Dan Brown
...,...,...,...,...,...,...,...,...,...,...
1408,948,554,Twilight (Twilight #1),501,2006-09-06,176,948,3,554,Stephenie Meyer
1409,948,554,Twilight (Twilight #1),501,2006-09-06,176,948,3,554,Stephenie Meyer
1410,948,554,Twilight (Twilight #1),501,2006-09-06,176,948,5,554,Stephenie Meyer
1411,948,554,Twilight (Twilight #1),501,2006-09-06,176,948,4,554,Stephenie Meyer


In [25]:
func_query(
    """
    SELECT AVG(rating) AS average_rating, author
    FROM
    (SELECT *
        FROM
        books
        INNER JOIN (SELECT book_id, rating
        FROM
        ratings
        WHERE book_id IN (SELECT book_id
                        FROM
                        ratings
                        GROUP BY book_id
                        HAVING COUNT(rating) > 50 OR COUNT(rating) = 50)) AS SQBT ON SQBT.book_id = books.book_id
        
    ) AS D_SQBT
    INNER JOIN authors ON D_SQBT.author_id = authors.author_id
   GROUP BY author
   ORDER BY average_rating DESC
    """
)

,average_rating,author
0,4.287097,J.K. Rowling/Mary GrandPré
1,4.264151,Markus Zusak/Cao Xuân Việt Khương
2,4.246914,J.R.R. Tolkien
3,4.192308,Louisa May Alcott
4,4.080645,Rick Riordan
5,3.901408,William Golding
6,3.825581,J.D. Salinger
7,3.789474,Paulo Coelho/Alan R. Clarke/Özdemir İnce
8,3.787879,William Shakespeare/Paul Werstine/Barbara A. M...
9,3.750000,Lois Lowry


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré. Средняя оценка 4,287 (средняя средних - 4,284).

# 5. Определение среднего количества обзоров от пользователей, которые поставили больше 50 оценок

Сначала нужно определить `username` полльзователй, которые поставили больше 50 оценок:

In [26]:
func_query(
    """
    SELECT username, COUNT('rating_id') AS ratings_count
    FROM
    ratings
    GROUP BY username
    HAVING COUNT('rating_id') > 50
    """
)

,username,ratings_count
0,sfitzgerald,55
1,jennifermiller,53
2,xdavis,51
3,paul88,56
4,martinadam,56
5,richard89,55


Из таблицы с обзорами отбираются только те, что сделали пользователи из полученного списка:

In [27]:
func_query(
    """
    SELECT *
    FROM
    reviews
    WHERE username IN (SELECT username
                        FROM
                        ratings
                        GROUP BY username
                        HAVING COUNT('rating_id') > 50)
    """
)

,review_id,book_id,username,text
0,16,6,sfitzgerald,Miss effort gun strong surface civil. Resource...
1,20,8,richard89,Sister statement than reflect. Crime hot guess...
2,30,11,richard89,Far visit rich describe. Heart blue generation...
3,40,15,martinadam,Deal fund decision management million black me...
4,51,18,jennifermiller,Lose act beautiful. Drive responsibility every...
...,...,...,...,...
141,2743,982,paul88,Station during food foot every end hour. Job o...
142,2750,985,jennifermiller,Prove instead wrong town. Police everybody hav...
143,2773,994,sfitzgerald,Concern lawyer a quality fish reason not. Agen...
144,2782,997,richard89,Step show wait answer range adult. Cell oil al...


Количество обзоров, сделанных каждым пользователем из списка:

In [28]:
func_query(
    """
    SELECT username, COUNT(review_id) AS reviews_count
    FROM
    reviews
    WHERE username IN (SELECT username
                        FROM
                        ratings
                        GROUP BY username
                        HAVING COUNT('rating_id') > 50)
    GROUP BY username
    """
)

,username,reviews_count
0,sfitzgerald,28
1,jennifermiller,25
2,xdavis,18
3,paul88,22
4,martinadam,27
5,richard89,26


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок:

In [29]:
func_query(
    """
    SELECT AVG(reviews_count) AS avg_count
    FROM
        (SELECT username, COUNT(review_id) AS reviews_count
        FROM
        reviews
        WHERE username IN (SELECT username
                        FROM
                        ratings
                        GROUP BY username
                        HAVING COUNT('rating_id') > 50)
        GROUP BY username) AS SQBT
    """
)

,avg_count
0,24.333333
